This notebook is about __Random Forest__ performance in __full phase__

In [1]:
from models import random_forest_pcc
from common import *

In [2]:
# estimators here
full_estimators = [random_forest_pcc]#, linear_l1_pcc, linear_pcc, glm_l1_pcc, glm_l2_pcc]
full_names = ['linear_non_pcc']#, 'linear_l1_pcc', 'linear_l2_pcc', 'glm_l1_pcc', 'glm_l2_pcc']

In [3]:
print('Interpolation')
print("Current Prediction Order is:", data.y.columns.tolist())
print('Num of predictors:, ', data.x1.shape[1])

interpolation = Experiment(full_estimators, 
                            full_names,
                            KFold(30, shuffle=True, random_state=STATE),
                            data.x1, data.y.replace(-1.0, 0.0),
                            groups=data.comp_ids.array, 
                            evaluators=[hamming_loss, ErrorEvaluator, NegLogLikelihoodEvaluator(base=2)],
                            verbose=True).run()

Interpolation
Current Prediction Order is: ['sphere', 'worm', 'vesicle', 'other']
Num of predictors:,  23
Running experiment with 30 repetitions
******************************


In [4]:
interpolation.summary()

,mean_train_hamming loss,std_train_hamming loss,mean_test_hamming loss,std_test_hamming loss,mean_train_error,std_train_error,mean_test_error,std_test_error,mean_train_log likelihood,std_train_log likelihood,mean_test_log likelihood,std_test_log likelihood
linear_non_pcc,0.000204,0.000425,0.066294,0.028894,0.000466,0.000909,0.204386,0.074076,-0.246837,0.003722,-1.211992,0.916856
